## 1. Installation

In [1]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.31
    Uninstalling langchain-core-0.3.31:
      Successfully uninstalled langchain-core-0.3.31
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15


## 2. Setup the Enviroment

In [2]:
import os

In [3]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

##3: Large Language Models

The basic building block of LangChain is a Large Language Model which takes text as input and generates more text

Suppose we want to generate a company name based on the company description, so we will first initialize an OpenAI wrapper. In this case, since we want the output to be more random, we will intialize our model with high temprature.

The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

temperature value--> how creative we want our model to be

0 ---> temperature it means model is  very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

A generic interface for all LLMs. See all LLM providers: https://python.langchain.com/en/latest/modules/models/llms/integrations.html

## Hugging Face

In [4]:
!pip install huggingface_hub

In [5]:
from langchain import HuggingFaceHub

In [6]:
llm = HuggingFaceHub(repo_id = 'google/flan-t5-large', model_kwargs = {"temperature":0, "max_length":64})

<ipython-input-6-ccb21efac1ec>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id = 'google/flan-t5-large', model_kwargs = {"temperature":0, "max_length":64})


In [7]:
llm("Translate English to German : How are you?")

<ipython-input-7-d44204f1b99c>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("Translate English to German : How are you?")


'Wie sind Sie?'

## Prompt

In [8]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("what is good name for company that makes {products}")
#prompt.format(products = "colourful socks")

## Chain

In [9]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt)
responce = chain.run('football')
print(responce)

<ipython-input-9-be0c5a60a881>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt = prompt)
<ipython-input-9-be0c5a60a881>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  responce = chain.run('football')


Adidas


### Simple Sequential Chain

In [10]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [11]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("indian")
print(content)

A burger


### Sequential Chain

In [12]:
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [13]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [14]:
print(chain({"cuisine": "indian"}))

{'cuisine': 'indian', 'restaurant_name': 'Indian restaurant', 'menu_items': 'A burger'}


<ipython-input-14-33865b5de443>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain({"cuisine": "indian"}))


##07: Memory

Chatbot application like ChatGPT, you will notice that it remember past information

###ConversationBufferMemory

We can attach memory to remember all previous conversation

In [15]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

memory = ConversationBufferMemory()

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)

<ipython-input-15-092188f73f32>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


Mexican restaurant


In [16]:
name = chain.run("Arabic")
print(name)

arame


In [17]:
print(chain.memory.buffer)

Human: Mexican
AI: Mexican restaurant
Human: Arabic
AI: arame


###ConversationChain

Conversation buffer memory goes growing endlessly

Just remember last 5 Conversation Chain

Just remember last 10-20 Conversation Chain

In [18]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=llm)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<ipython-input-18-bb6684f844b9>:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(llm=llm)


In [19]:
convo.run("Who won the first cricket world cup?")

'The first cricket world cup was won by India.'

In [20]:
convo.run("How much is 5+5?")

'5+5 is a number that is repeated indefinitely.'

In [21]:
convo.run("Who was the captain of the winning team?")

'The captain of the winning team was Virat Kohli.'

In [22]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI: The first cricket world cup was won by India.
Human: How much is 5+5?
AI: 5+5 is a number that is repeated indefinitely.
Human: Who was the captain of the winning team?
AI: The captain of the winning team was Virat Kohli.


###ConversationBufferWindowMemory

In [23]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

convo = ConversationChain(
    llm=llm,
    memory=memory
)
convo.run("Who won the first cricket world cup?")

<ipython-input-23-c44103b4dc92>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3)


'The first cricket world cup was won by India.'

In [24]:
convo.run("How much is 5+5?")

'5+5 is a number that is repeated indefinitely.'

In [25]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI: The first cricket world cup was won by India.
Human: How much is 5+5?
AI: 5+5 is a number that is repeated indefinitely.
